# 데이터 불러오기

In [29]:
data_path = "./skeleton_estimation_train/skeleton_train_data"

# Train

In [30]:
batch_size = 10
train_size = 10000

In [39]:
import lmdb
from tensorpack.dataflow import *
from tensorpack.dataflow.common import BatchData, MapData
import numpy as np

print ("Loading the COCO data set ...")
df = lmdb.open(data_path)
df = BatchData(df, batch_size, use_list=False)
df = MapData(df, lambda x: ( [x[0], x[1], x[2]],
                              [x[3], x[4], x[3], x[4], x[3], x[4], x[3], x[4], x[3], x[4], x[3], x[4]])
                            )
df.reset_state()
print("Loaded {} training samples!".format(train_size))

Loading the COCO data set ...


TypeError: object of type 'Environment' has no len()

In [5]:
def generate_pose_detection_model(n_stage):
    # build the input
    inputs = []
    
    img_in = Input( shape = (None, None, 3))
    inputs.append(img_in)
    w_vector_field_in = Input( shape = (None, None, 38))
    inputs.append(w_vector_field_in)
    w_confi_in = Input( shape = (None, None, 19))
    inputs.append(w_confi_in)

    # process the input image
    img_processed = Lambda(lambda x: x / 256 - 0.5)(img_in)

    #run pure vgg19
    vgg_out = vgg_19(img_processed)

    # run state 1
    stage_1_b1_out = pose_model_stage_1(vgg_out, 38, 1)
    w_1_out = Multiply(name = "weight_stage1_L1") ( [stage_1_b1_out, w_vector_field_in])

    # run state 1
    stage_1_b2_out = pose_model_stage_1(vgg_out, 19, 2)
    w_2_out = Multiply(name = "weight_stage1_L2") ( [stage_1_b2_out, w_confi_in])

    x = Concatenate()([stage_1_b1_out, stage_1_b2_out, vgg_out])
    # build output
    outputs = []
    outputs.append(w_1_out)
    outputs.append(w_2_out)

    for stage_id in range(2,n_stage + 1):
        stage_2_b1_out = pose_model_stage_2(x, 38, 1, stage_id)
        w_1_out = Multiply(name = "weight_stage{}_L1".format(stage_id)) ( [stage_2_b1_out, w_vector_field_in])

        stage_2_b2_out = pose_model_stage_2(x, 19, 2, stage_id)
        w_2_out = Multiply(name = "weight_stage{}_L2".format(stage_id)) ( [stage_2_b2_out, w_confi_in])

        outputs.append(w_1_out)
        outputs.append(w_2_out)

        if (stage_id < n_stage):
            x = Concatenate()([stage_2_b1_out, stage_2_b2_out, vgg_out])
    # model = Model( inputs = inputs, outputs = outputs)
    model = Model(inputs=[img_in], outputs = [stage_2_b1_out, stage_2_b2_out])
    return model

In [7]:
from tensorflow.keras.applications.vgg19 import VGG19

def load_keras_vgg_weights(model):
    vgg_name = {
        'conv1_1': 'block1_conv1',
        'conv1_2': 'block1_conv2',
        'conv2_1': 'block2_conv1',
        'conv2_2': 'block2_conv2',
        'conv3_1': 'block3_conv1',
        'conv3_2': 'block3_conv2',
        'conv3_3': 'block3_conv3',
        'conv3_4': 'block3_conv4',
        'conv4_1': 'block4_conv1',
        'conv4_2': 'block4_conv2'
    }
    keras_vgg = VGG19(include_top=False, weights='imagenet')
    for layer in model.layers:
        if layer.name in vgg_name:
            layer.set_weights(keras_vgg.get_layer(vgg_name[layer.name]).get_weights())

In [8]:
def gen(df):
    while True:
        for i in df.get_data():
            yield i

In [9]:
def get_loss_funcs():
    def _eucl_loss(x, y):
        return K.sum(K.square(x - y)) / 10 / 2

    losses = {}
    losses["weight_stage1_L1"] = _eucl_loss
    losses["weight_stage1_L2"] = _eucl_loss
    losses["weight_stage2_L1"] = _eucl_loss
    losses["weight_stage2_L2"] = _eucl_loss
    losses["weight_stage3_L1"] = _eucl_loss
    losses["weight_stage3_L2"] = _eucl_loss
    losses["weight_stage4_L1"] = _eucl_loss
    losses["weight_stage4_L2"] = _eucl_loss
    losses["weight_stage5_L1"] = _eucl_loss
    losses["weight_stage5_L2"] = _eucl_loss
    losses["weight_stage6_L1"] = _eucl_loss
    losses["weight_stage6_L2"] = _eucl_loss

    return losses

In [ ]:
model = generate_pose_detection_model(6)
print("Prepare to train a new model")
load_keras_vgg_weights(model)
model.compile(loss='categorical_crossentropy', optimizer = 'sgd', metrics=["accuracy"])
model.fit_generator(gen(df), steps_per_epoch=100, epochs=2, use_multiprocessing=False, initial_epoch=0)